In [ ]:
%pip install folium

In [ ]:
import folium
#m = folium.Map()
#m

In [ ]:
import pandas as pd
import re

In [ ]:
finddigitalreg=lambda x :re.findall(r"\d+",x)

In [ ]:
#Data : https://en.wikipedia.org/wiki/List_of_best-selling_singles
df=pd.read_csv("music.csv",header=None,names=["Artist","Single","Year","Sales","Source"])
df["NumberSales"]=df["Sales"].apply(lambda x: finddigitalreg(x)[0]).astype('int32')
df["Artists"]=df["Artist"].apply(lambda x: x.split(" featuring ") if x.split("featuring") else "")
df["MainArtist"]=df["Artists"].apply(lambda x: x[0])
df["FeatArtist"]=df["Artists"].apply(lambda x: x[1] if len(x)==2 else None)
df["TenYears"]=df["Year"].apply(lambda x: str(x)[2]+"0")
df=df.drop(columns=["Source","Sales","Artist","Artists","Year"])
df

In [ ]:
df_artist=pd.read_csv("artist.csv")
df_artist["Artist"]=df_artist["Artist"].apply(lambda x: x.replace("'",""))

In [ ]:
df_artist

In [ ]:
df=df.merge(df_artist,how="inner",left_on="MainArtist",right_on="Artist")
df

In [ ]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [ ]:
df_group_by=df.groupby("Born").sum(numeric_only=True)
df_group_by=df_group_by.reset_index()
df_group_by_double_countries=df_group_by[df_group_by.Born.str.contains("\+")].copy()
df_group_by=df_group_by[~df_group_by.Born.str.contains("\+")]
df_group_by.sort_values(by="NumberSales")
df_group_by

In [ ]:
for i,j in df_group_by_double_countries.iterrows():
    country1,country2=j.Born.split("+")
    
    df_group_by=pd.concat(
        [
            df_group_by,
            pd.DataFrame(
                {
                    "Born": [country1],
                    "NumberSales": [j.NumberSales/2],
                }
            ),
        ]
    )
    df_group_by=pd.concat(
        [
            df_group_by,
            pd.DataFrame(
                {
                    "Born": [country2],
                    "NumberSales": [j.NumberSales/2],
                }
            ),
        ]
    )
df_group_by=df_group_by.groupby(by="Born").sum(numeric_only=True).reset_index()
df_group_by

In [ ]:
df_group_by.replace('USA', "United States of America", inplace = True)
df_group_by.replace('UK', "United Kingdom", inplace = True)

In [ ]:
import folium


m = folium.Map([0,0], zoom_start=2)

folium.Choropleth(
    geo_data=country_shapes,
    #highlight=True, 

    fill_opacity=0.7,
    data=df_group_by,           
    bins=[0, 20, 40, 100, 1000],
    columns=['Born', 'NumberSales'],
    fill_color='PuRd',
    key_on="feature.properties.name",
    nan_fill_color='blue',
    name="Total of Copies of Best-selling physical singles by Country of birth",
).add_to(m)
m